# %% [markdown]
# # 🚀 FaceAge Web Interface
# ### Загрузите фото → Получите возраст лица
# ---

# %% [markdown]
# ## ⚙️ Установка необходимых компонентов

In [ ]:

!pip install -q streamlit pyngrok mtcnn opencv-python pillow numpy tensorflow

# %% [markdown]
# ## 📁 Создаем файл приложения
%%writefile faceage_app.py

In [ ]:

import streamlit as st
import numpy as np
import cv2
from PIL import Image
from mtcnn import MTCNN
import tensorflow as tf
from tensorflow.keras.models import load_model
import urllib.request
import os

# Настройки
st.set_page_config(page_title="FaceAge", layout="wide")
st.markdown("""
<style>
    .reportview-container {background: #f0f2f6}
    .big-font {font-size:24px !important; color: #2a3f5f}
    .result-box {border-radius:10px; padding:20px; background:#ffffff; margin-top:20px}
</style>
""", unsafe_allow_html=True)

# Заголовок
st.image("https://raw.githubusercontent.com/AIM-Harvard/FaceAge/main/docs/logo.png", width=200)
st.title("🔍 FaceAge: Определение возраста по фото")
st.markdown("Загрузите фотографию лица, и алгоритм предскажет возраст", class_="big-font")

# Загрузка модели
@st.cache_resource
def load_model_from_github():
    model_url = "https://github.com/AIM-Harvard/FaceAge/raw/main/models/faceage_model.h5"  # Убедитесь в правильности URL
    model_path = "faceage_model.h5"
    if not os.path.exists(model_path):
        urllib.request.urlretrieve(model_url, model_path)
    return load_model(model_path)

try:
    model = load_model_from_github()
except Exception as e:
    st.error(f"Ошибка загрузки модели: {str(e)}")
    st.stop()

# Детектор лиц
detector = MTCNN()

def process_image(img):
    # Конвертация в RGB
    img_rgb = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)

    # Детекция лица
    faces = detector.detect_faces(img_rgb)
    if not faces:
        return None

    # Выбираем наибольшее лицо
    main_face = max(faces, key=lambda x: x['confidence'])
    x, y, w, h = main_face['box']

    # Обрезка и ресайз
    face_img = img_rgb[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (128, 128))
    face_img = face_img.astype('float32') / 255.0

    return face_img, (x, y, w, h)

# Интерфейс загрузки
uploaded_file = st.file_uploader("", type=["jpg", "jpeg", "png"])

col1, col2 = st.columns(2)

if uploaded_file:
    with col1:
        st.header("Исходное изображение")
        image = Image.open(uploaded_file)
        st.image(image, use_column_width=True)

    with st.spinner('🔍 Анализируем лицо...'):
        result = process_image(image)

    if result:
        face_img, bbox = result

        with col2:
            st.header("Результат")
            st.image(face_img, caption="Обнаруженное лицо", clamp=True, channels='RGB')

            # Предсказание
            age = model.predict(np.expand_dims(face_img, axis=0))[0][0]

            st.markdown(f"""
            <div class="result-box">
                <h3>Результат анализа:</h3>
                <p style='font-size:36px; color:#2a3f5f; font-weight:bold;'>
                    {age:.1f} лет
                </p>
            </div>
            """, unsafe_allow_html=True)
    else:
        st.error("Не удалось обнаружить лицо. Попробуйте другое фото.")

# %% [markdown]
# ## 🚀 Запуск веб-приложения

In [ ]:

from pyngrok import ngrok

# Запуск Streamlit в фоне
!streamlit run faceage_app.py --server.port 8501 &>/dev/null&

# Создание публичной ссылки
public_url = ngrok.connect(addr='8501')
print(f"\n\n✨ Ваше приложение доступно по ссылке: {public_url}\n")